In [1]:
import os
import os.path

In [12]:
for dirpath, dirnames, filenames in os.walk("./Albums"):
    for filename in [f for f in filenames if f.endswith(".txt")]:
        print(dirpath)
        print (os.path.join(dirpath, filename))
        print('###########')

./Albums/album2
./Albums/album2/a2_t1.txt
###########
./Albums/album2
./Albums/album2/a2_t2.txt
###########
./Albums/album1
./Albums/album1/a1_t1.txt
###########
./Albums/album1
./Albums/album1/a1_t2.txt
###########
